In [ ]:
# Ran this on the selfnet data and then put it in the test script

import os
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread, imsave
from skimage import exposure
from skimage.restoration import denoise_wavelet
from skimage.filters import median
import stackview

# ===============================
# INPUT FILE PATH
# ===============================
image_path = r"C:\Users\rexsw\Desktop\JHU\BDD\Dendrite_2p_20250925\New data2\raw\F13_2_20250323_roi1_Red_shifted.tif"
assert os.path.exists(image_path), f"File not found: {image_path}"

print(">>> Loading entire TIFF (3D)...")
image = imread(image_path)  # Expect shape (Z, Y, X)

assert image.ndim == 3, f"Expected 3D TIFF. Got shape: {image.shape}"


>>> Loading entire TIFF (3D)...


In [2]:
# stackview.slice(image)

In [ ]:


# # ===============================
# # GLOBAL NORMALIZATION
# # ===============================
print(">>> Global normalization...")
vmin = np.percentile(image, 0.5)
vmax = np.percentile(image, 99.5)
image_norm = np.clip((image - vmin) / (vmax - vmin), 0, 1)

# Convert to 12-bit range for processing stability
image_uint = (image_norm * 1024).astype(np.uint16)

i1 = image_uint
i2 = image_uint
i3 = image_uint
i4 = image_uint

>>> Global normalization...


In [4]:
# stackview.histogram(image_uint)

In [28]:

# ===============================
# GLOBAL CLAHE (gentle)
# ===============================
print(">>> Applying global CLAHE...")
image_clahe = exposure.equalize_adapthist(
    image_uint, 
    clip_limit=0.005 #0.005  # ↓ keep noise controlled
)

i1_clahe = exposure.equalize_adapthist(
    i1, 
    clip_limit=0.05 #0.005  # ↓ keep noise controlled
)

i2_clahe = exposure.equalize_adapthist(
    i2, 
    clip_limit=0.5 #0.005  # ↓ keep noise controlled
)


>>> Applying global CLAHE...


In [ ]:
stackview.histogram(image_clahe)


In [23]:

stackview.histogram(i1_clahe)


In [24]:

stackview.histogram(i2_clahe)

In [7]:

# ===============================
# WAVELET DENOISING
# ===============================
print(">>> Wavelet denoise (BayesShrink, soft)...")
denoised = denoise_wavelet(
    image_clahe,
    channel_axis=None,
    method='BayesShrink',
    mode='soft',
    wavelet='db2',
    rescale_sigma=True
)

i1_dn1 = denoise_wavelet(
    i1_clahe,
    channel_axis=None,
    method='BayesShrink',
    mode='soft',
    wavelet='db2',
    rescale_sigma=True
)


>>> Wavelet denoise (BayesShrink, soft)...


In [8]:
# stackview.histogram(denoised)

In [9]:

# ===============================
# MEDIAN SMOOTHING
# ===============================
print(">>> Median filtering per slice...")
denoised = (denoised * 1024).astype(np.uint16)
filtered = np.array([
    median(denoised[z], footprint=np.ones((3,3)))  # 3x3 smoothing
    for z in range(denoised.shape[0])
])

i1_dn2 = np.array([
    median(i1_clahe[z], footprint=np.ones((3,3)))  # 3x3 smoothing
    for z in range(i1_clahe.shape[0])
])

>>> Median filtering per slice...


In [19]:

# ===============================
# ROBUST PERCENTILE CONTRAST STRETCH
# (less aggressive → preserves faint signals)
# ===============================
print(">>> Adaptive percentile stretch...")
low = np.percentile(filtered, 78)     # tuneable
high = np.percentile(filtered, 99.9)  # tuneable

final = np.clip((filtered - low) / (high - low), 0, 1)
final = (final * 4095).astype(np.uint16)  # convert to 12-bit TIFF depth
I = final

low1 = np.percentile(i1_dn1, 78)     # tuneable
high1 = np.percentile(i1_dn1, 99.9)  # tuneable

final1 = np.clip((i1_dn1 - low1) / (high1 - low1), 0, 1)
final1 = (final1 * 4095).astype(np.uint16)  # convert to 12-bit TIFF depth
I1 = final1

low12 = np.percentile(i1_dn2, 78)     # tuneable
high12 = np.percentile(i1_dn2, 99.9)  # tuneable

final12 = np.clip((i1_dn2 - low12) / (high12 - low12), 0, 1)
final12 = (final12 * 4095).astype(np.uint16)  # convert to 12-bit TIFF depth
I12 = final12

low3 = np.percentile(i3, 78)     # tuneable
high3 = np.percentile(i3, 99.9)  # tuneable

final3 = np.clip((i3 - low3) / (high3 - low3), 0, 1)
final3 = (final3 * 4095).astype(np.uint16)  # convert to 12-bit TIFF depth
I3 = final3


>>> Adaptive percentile stretch...


In [11]:
# stackview.histogram(denoised)

In [12]:

# ===============================
# SAVE RESULTS
# ===============================
# save_dir = r"C:\Users\rexsw\Desktop\JHU\BDD\SpineDetect-joey-branch\processed"
# os.makedirs(save_dir, exist_ok=True)

# save_path = os.path.join(save_dir, "F13_2_20250323_roi1_Red_shifted_contrast_clahe_wavelet_med.tif")
# imsave(save_path, filtered)
# print(f"✅ Saved processed image: {save_path}")

# ===============================
# QUICK VISUAL CHECK (middle slice)
# ===============================
# mid = final.shape[0] // 2
# plt.figure(figsize=(10,5))
# plt.title("Final Processed (Middle Slice)")
# plt.imshow(final[mid], cmap="gray")
# plt.axis("off")
# plt.show()

print("🎉 Preprocessing completed successfully!")


🎉 Preprocessing completed successfully!


In [27]:
import stackview
stackview.curtain(I3, I)

In [ ]:
# plt.figure(figsize=(6,4))
# plt.hist(image_clahe.ravel(), bins=256, color='gray', alpha=0.8)
# plt.title("Intensity Histogram")
# plt.xlabel("Pixel Intensity")
# plt.ylabel("Frequency")
# plt.show()